In [2]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt

In [16]:
from scipy import io as spio
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    
    from: `StackOverflow <http://stackoverflow.com/questions/7008608/scipy-io-loadmat-nested-structures-i-e-dictionaries>`_
    '''
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, spio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict

In [9]:
def GetUniqueList(files):
    # get unique flyIDs

    flyIDs = []
    for f in files:
        a, b = os.path.basename(f).split('_', 1)
        flyIDs.append(a)

    # insert the list to the set 
    list_set = set(flyIDs) 
    # convert the set to the list 
    unique_list = (list(list_set))
    return(unique_list)

In [12]:
def organizeData(eachFly, Exp):
    files = glob.glob(direc + eachFly + "*.mat")
    
    if Exp in ['Slit_sc', 'asymmetric_wings']:
        bodyparts = ['LW', 'RW']
    elif Exp in ['epi_ridge_cut', 'Haltere_loading']:
        bodyparts = ['LW', 'RW', 'LH', 'RH']
    
    df = pd.DataFrame(data = None)
    for f in files:
        val = []
        body = []
        n = []
        trt = []
        exp = []
        l = []

        [flyID, experiment, treatment, _] = os.path.basename(f).split('_')

        mat_struct = loadmat(f)
    #     mat_struct.keys()

        for i, b in enumerate(bodyparts):
            temp = mat_struct['wba'][b]
            val.extend(temp)
            l = len(temp)
            body.extend([b]*l)
            exp.extend([experiment]*l)
            n.extend([flyID]*l)
            trt.extend([treatment]*l)

        # new_list = boolean_indexing(listlist)

        subdf = pd.DataFrame({'body': body,
                           'amplitude': val,
                           'flyID': n, 
                           'experiment': exp,
                           'treatment': trt
                          })
        df = df.append(subdf)
    return(df, flyID)

In [15]:
Exp = ['asymmetric_wings', 'Slit_sc', 'epi_ridge_cut', 'Haltere_loading']

for exp in Exp:
    direc = r"../dataFolder/AmplitudeAnalysis/v1/" + exp + "/Kinematics/"
    files = glob.glob(direc + "fly*.mat")
    unique_list = GetUniqueList(files)
    for eachFly in unique_list:
    `    df, flyID = organizeData(eachFly, exp)
        df.to_csv('../dataFolder/AmplitudeAnalysis/v1/' + exp + '/' + flyID + '_AmpDataCompiled.csv')

### Add winglength to the wing clipping data

### intact and slit Sc

In [13]:
direc = r"../dataFolder/Submission 3/Final Submission/Data to be uploaded to Dryad/Figure2/"
file = pd.read_csv(direc + 'merged_TD', sep = "\t")
data = file.loc[:,('Treatment', 'Fly', 'Cut' ,'wing_mean.Corleft', 'Average.Corleft', 'Average.Corright')]

In [51]:
data['Cut'] = data['Cut'].replace({'Cut1':'cutwing1',
                               'Cut2':'cutwing2',
                               'Cut3': 'cutwing3',
                               'Cut4': 'cutwing4',
                               'Cut5': 'cutwing5',
                               'Intact': 'intactwings'
                              })
data['Treatment'] = data['Treatment'].replace({'Intact': 'asymmetric_wings',
                                               'Slit': 'Slit_sc'})
data['Fly'] = data['Fly'].str.lower()
data.head()

,Treatment,Fly,Cut,wing_mean.Corleft,Average.Corleft,Average.Corright
0,asymmetric_wings,fly1,cutwing1,0.821102,93.154528,93.349334
1,asymmetric_wings,fly1,cutwing2,0.666827,101.806114,102.028900
2,asymmetric_wings,fly1,cutwing3,0.573657,108.858352,108.810557
3,asymmetric_wings,fly1,cutwing4,0.461742,93.192249,92.950641
4,asymmetric_wings,fly1,cutwing5,0.391743,84.506527,83.927994


In [60]:
Exp = ['asymmetric_wings', 'Slit_sc']

for exp in Exp:
    direc = r"../dataFolder/AmplitudeAnalysis/v1/" + exp + '/'
    files= glob.glob(direc + '*_AmpDataCompiled.csv')
    for f in files:
        Edit = pd.read_csv(f)
        if 'winglength' not in Edit:
            flyID = Edit.flyID.unique()[0]

            subset = data.loc[(data.Fly  == flyID) & (data.Treatment == exp), :]
            mapping_dict = subset.loc[:, ['Cut', 'wing_mean.Corleft']].set_index('Cut').to_dict()['wing_mean.Corleft']
            Edit['winglength'] = Edit['treatment'].map(mapping_dict)
            Edit.to_csv('../dataFolder/AmplitudeAnalysis/v1/' + exp + '/' + flyID + '_AmpDataCompiled.csv')

### for epi ridge

In [63]:
data = pd.read_csv("../dataFolder/Submission 3/Final Submission/Data to be uploaded to Dryad/EpiRidgeCut_Figure3/compiledDataForAggregratePlots.csv")
data.treatment = data.treatment.str.split('_', expand = True)[0]
data.head()

,Unnamed: 0,flyID,treatment,winglength,wingHz,leftHalHz,rightHalHz,binned
0,0,fly1,cutwing1,0.82,138.095238,138.095238,138.095238,"(0.7, 0.9]"
1,1,fly1,cutwing2,0.72,132.321429,132.797619,132.310924,"(0.7, 0.9]"
2,2,fly1,cutwing3,0.58,151.648352,151.904762,151.831502,"(0.5, 0.6]"
3,3,fly1,cutwing4,0.50,138.387470,130.766537,137.388943,"(0.4, 0.5]"
4,4,fly1,cutwing5,0.38,187.272727,124.550654,125.486992,"(0.3, 0.4]"


In [72]:
exp = 'epi_ridge_cut'

direc = r"../dataFolder/AmplitudeAnalysis/v1/" + exp + '/'
files= glob.glob(direc + '*_AmpDataCompiled.csv')
for f in files:
    Edit = pd.read_csv(f)
    if 'winglength' not in Edit:
        flyID = Edit.flyID.unique()[0]

        subset = data.loc[data.flyID  == flyID, :]
        mapping_dict = subset.loc[:, ['treatment', 'winglength']].set_index('treatment').to_dict()['winglength']
        Edit['winglength'] = Edit['treatment'].map(mapping_dict)
        Edit.to_csv('../dataFolder/AmplitudeAnalysis/v1/' + exp + '/' + flyID + '_AmpDataCompiled.csv')

In [141]:
# add winglength for fly 8 and fly 9
# use first 2 wingbeats (~20 frames) to compute the winglength for each treatment
fly = ['fly8', 'fly9']
body = ['LW', 'RW']

for fly in fly:
    files = glob.glob('../dataFolder/AmplitudeAnalysis/v1/epi_ridge_cut/Kinematics/' + fly + '_epi_*_Kinematics.mat')
    winglength = []
    trt = []
    for f in files:
        _,_, treatment, _ = os.path.basename(f).split('_')
        wl =[]
        df = loadmat(f)
        vectors = df['rotated']
        
        for b in body:
            temp = pd.DataFrame(data = vectors[b],
                              columns = ['x', 'y', 'z'])
            length =np.linalg.norm(temp, axis = 1)
            temp['winglength'] = length
            wl.append(np.nanmean(temp.loc[0:20, 'winglength']))
        
        l = np.nanmean(wl)
        winglength.append(l)
        trt.append(treatment)
    var=pd.DataFrame({'treatment':trt, 'winglength':winglength})
    longest = var.loc[var['treatment'] == 'intactwings', 'winglength'].values
    var['winglength'] = var['winglength']/longest
    var.to_csv('../dataFolder/AmplitudeAnalysis/v1/epi_ridge_cut/' + fly + '_winglength.csv')

In [142]:
direc = r"../dataFolder/AmplitudeAnalysis/v1/" + 'epi_ridge_cut' + '/'
fly = ['fly8', 'fly9']
for fly in fly:
    s_files= pd.read_csv(direc + fly + '_AmpDataCompiled.csv')
    length = pd.read_csv(direc + fly + '_winglength.csv')
    mapping_dict = length.set_index('treatment').to_dict()['winglength']
    s_files['winglength'] = s_files['treatment'].map(mapping_dict)
    s_files.to_csv(direc + fly + '_AmpDataCompiled.csv')